In [ ]:
import tensorflow as tf
from tensorflow.keras.regularizers import l1
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split

X = np.random.random((100000, 2)) * 2 - 1
y = ((np.sin(X[:, 1] * np.pi * X[:,1] * 6) + np.cos(X[:, 0] * np.pi * X[:, 0] * 4)) > 0).astype(int)

# randomly split the data into training, cross validation and test sets
random_state = 42

X_train, X_blind, y_train, y_blind = train_test_split(
    X, y, test_size=0.2, random_state=random_state)

X_test, X_cv, y_test, y_cv = train_test_split(
    X_blind, y_blind, test_size=0.5, random_state=random_state)

print(f'tf.__version__={tf.__version__}')
print(f'X.shape={X.shape}')
print(f'y.shape={y.shape}')
print(f'y max={np.max(y)} y min={np.min(y)}')

In [ ]:
plt.figure(figsize=(13, 4))

plt.subplot(1, 3, 1)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=1, cmap=plt.cm.Spectral)
plt.title('Training Set')

plt.subplot(1, 3, 2)
plt.scatter(X_cv[:, 0], X_cv[:, 1], c=y_cv, s=1, cmap=plt.cm.Spectral)
plt.title('Validation Set')

plt.subplot(1, 3, 3)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=1, cmap=plt.cm.Spectral)
plt.title('Test Set')

plt.show()


In [ ]:
# setup the layers
regfac = 0.00005
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=l1(regfac)),
    tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=l1(regfac)),
    tf.keras.layers.Dense(2, activation='sigmoid', kernel_regularizer=l1(regfac))
])

epochs = 200
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [4]:
%%time
history = model.fit(X_train, y_train, epochs=epochs, verbose=1, validation_data=(X_cv, y_cv))
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.9092 - loss: 0.2514 - val_accuracy: 0.9121 - val_loss: 0.2419
Epoch 33/200
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9092 - loss: 0.2496 - val_accuracy: 0.9036 - val_loss: 0.2465
Epoch 34/200
 227/2500 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9050 - loss: 0.2540

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

print(f'test acc: {test_acc*100:0.2f}% test loss: {test_loss:0.3f}')

In [ ]:
V1 = np.linspace(-1, 1, 150)
V2 = np.linspace(-1, 1, 150)
V1, V2 = np.meshgrid(V1, V2)
XT = np.c_[V1.ravel(), V2.ravel()]
Z = model.predict(XT)
YT = np.argmax(Z, axis=1)


In [ ]:
plt.figure(figsize=(13, 4))

plt.subplot(1, 3, 1)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=1, cmap=plt.cm.Spectral)
plt.title('Training Set')

plt.subplot(1, 3, 2)
plt.scatter(XT[:, 0], XT[:, 1], c=Z[:, 1], s=1, cmap=plt.cm.Spectral)
plt.title('Decision Boundary Soft')

plt.subplot(1, 3, 3)
plt.scatter(XT[:, 0], XT[:, 1], c=YT, s=1, cmap=plt.cm.Spectral)
plt.title('Decision Boundary Hard')

plt.show()